In [1]:
import sys
sys.path.append('/usr/local/lib/python3.4/dist-packages')

In [2]:
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

In [3]:
% cd Applications
% cd elasticsearch-1.7.1
!./bin/elasticsearch -d

/home/davidstap/anaconda3/Applications
/home/davidstap/anaconda3/Applications/elasticsearch-1.7.1


# Group Assignment  week 8

## Notebook made by   (If not filled in correctly: 0 pts for assignment)

__Name(s)__: 

__Student id(s)__ : 

### Pledge (taken from [Coursera's Honor Code](https://www.coursera.org/about/terms/honorcode) )



Put here a selfie with your photo where you hold a signed paper with the following text: (if this is team work, put two selfies here). The link must be to some place on the web, not to a local file. **Assignments without the selfies will not be graded and receive 0 points.**

> My answers to homework, quizzes and exams will be my own work (except for assignments that explicitly permit collaboration).

>I will not make solutions to homework, quizzes or exams available to anyone else. This includes both solutions written by me, as well as any official solutions provided by the course staff.

>I will not engage in any other activities that will dishonestly improve my results or dishonestly improve/hurt the results of others.

<img src='link to your selfie'/>

In [4]:
import gzip
from xml.dom.minidom import parse, parseString

# open and read gzipped xml file
infile = gzip.open('../../telegraaf/telegraaf-1951.xml.gz')
content = infile.read()

In [5]:
print(content[:1000])

b'<?xml version="1.0" encoding="UTF-8"?>\n<pm:KBroot xmlns:pm="http://www.politicalmashup.nl" recordfile="">\n<pm:root><pm:docinfo/><pm:meta><dc:date xmlns:dc="http://purl.org/dc/elements/1.1/">1951-07-02</dc:date><dc:subject xmlns:dc="http://purl.org/dc/elements/1.1/">artikel</dc:subject><dc:identifier xmlns:dc="http://purl.org/dc/elements/1.1/">ddd:011140123:mpeg21:p001:a0001</dc:identifier><dc:source xmlns:dc="http://purl.org/dc/elements/1.1/"><dc:source><pm:link pm:source="832675288" pm:description="De Telegraaf"/></dc:source></dc:source></pm:meta><pm:content pm:source="http://kranten.kb.nl/view/article/id/ddd:011140123:mpeg21:p001:a0001" pm:id="ddd:011140123:mpeg21:p001:a0001"><title pm:id="ddd:011140123:mpeg21:p001:a0001.t">Ook Noord-Korea en China willen praten over wapenstilstand RIDGWAY\'S OPROEP AANVAARD Ontmoeting bij Kaesong (aan 38ste) voorgesteld MAAR EERST OVER 10 \xc3\x80 14 DAGEN</title><text><p pm:id="ddd:011140123:mpeg21:p001:a0001.1">TOKIO, 2 Juli. \xe2\x80\x94 De o

In [7]:
import xmltodict, json

o = xmltodict.parse(content)

# use xmltodict to convert xml file to json (elasticsearch needs json input)
#make json dump
dump = json.(o) # '{"e": {"a": ["text", "text"]}}'

In [272]:
# GET DATE FROM ARTICLE
def get_date(i):
    return o['pm:KBroot']['pm:root'][i]['pm:meta']['dc:date']['#text']

In [333]:
# GET TITEL FROM ARTICLE
def get_title(i):
    try:
        return o['pm:KBroot']['pm:root'][i]['pm:content']['title']['#text']
    except (KeyError):
        return ''

In [443]:
# GET TEXT FROM ARTICLE
def get_text(i):
    #sometimes, articles have more than 1 #text field
    try:
        return o['pm:KBroot']['pm:root'][i]['pm:content']['text']['p']['#text']
    except (TypeError):
        try:
            text = ''
            for j in range(len(o['pm:KBroot']['pm:root'][i]['pm:content']['text']['p'])):
                text = text + o['pm:KBroot']['pm:root'][i]['pm:content']['text']['p'][j]['#text']
            return text
        except (TypeError):
            return ''
    except (KeyError):
        return ''
 

In [432]:
o['pm:KBroot']['pm:root'][2603]['pm:content']

OrderedDict([('@pm:source', 'http://kranten.kb.nl/view/article/id/ddd:011140134:mpeg21:p002:a0024'), ('@pm:id', 'ddd:011140134:mpeg21:p002:a0024'), ('title', OrderedDict([('@pm:id', 'ddd:011140134:mpeg21:p002:a0024.t')])), ('text', None)])

In [444]:
get_text(2603)

''

In [447]:


telegraaf = list()

for i in range(len(o['pm:KBroot']['pm:root'])):
    telegraaf.append({'_type':'article', '_index':'telegraaf', 'id':i, 'date':get_date(i), 'title':get_title(i), 'text':get_text(i)})

In [449]:
get_text(3000)

'De eerste dag werd in de Olym. piajollen een hevije strijd geleverd tussen de oude en nieuwe kampi. oen, resp. Koos de Jong en Markus. Beiden gaven geen haarbreed toe, maar het was ten slotte De Jong, die ziin „Zomerweelde" het eerst over de finish bracht. Ook de Zondagrace was voor hem. Zijn zoontje heeft ook blijkbaar het zeilersbloed geërfd, het behield namelijk in beiae wedstrijden de leiding in de jeugdklasse. gevolgd door Kuylaars. In de Sharpies was de Zaterdagwedstrijd nu eens voor T. Kraan, die in de boot van zijr. broer voer. Zondag was S. J. Kraan, in eigen schip, hem echter verre de baps. CruldeTiond leverde met twee twee. de prijzen lang geen onverdiensto\' lijke prestatie. In de Pampusklasse vprsloeg Van de Berg Zondag zowel de nationale kampioen Volker als Van Staveren, die men vooral op eigen terrein niet mag onderschatten. Rest nog te vermelden, dat de wedstrijden in ren sportieve sfeer werden venreild, hetseen wel hieruit blijkt, dat er niet één protest is binnengeko

In [ ]:
# NEXT STEP: DUMP TELEGRAAF LIST TO ELASTICSEARCH (SEE TUTORIAL NOTEBOOK)

In [ ]:
# make 1 big json object4
# store this in elasticsearch
# find document types (for facets)

# store these things in elasticsearch

In [318]:
len(o['pm:KBroot']['pm:root'])

37375

In [266]:
#create index telegraaf
es.indices.create('telegraaf')

{'acknowledged': True}

In [295]:
# delete index telegraaf
#es.indices.delete(index='telegraaf')

In [296]:
es.count(index='david')

{'_shards': {'failed': 0, 'successful': 5, 'total': 5}, 'count': 0}

In [292]:
HOST = 'http://localhost:9200/david'
es = Elasticsearch(hosts=[HOST])


query={
  "query": {
    "match": {'letters':'aj'}
  }
}




es.search(body=query)

{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': 'AVgCbyEUpwdMr7iZJqhC',
    '_index': 'david',
    '_score': 4.0445223,
    '_source': {'id': 34, 'letters': 'aJ'},
    '_type': 'foo'},
   {'_id': 'AVgCbwajpwdMr7iZJqfe',
    '_index': 'david',
    '_score': 3.9444387,
    '_source': {'id': 34, 'letters': 'aJ'},
    '_type': 'foo'},
   {'_id': 'AVgCbwaipwdMr7iZJqfE',
    '_index': 'david',
    '_score': 3.5902672,
    '_source': {'id': 8, 'letters': 'aj'},
    '_type': 'foo'},
   {'_id': 'AVgCbyEUpwdMr7iZJqgo',
    '_index': 'david',
    '_score': 3.5902672,
    '_source': {'id': 8, 'letters': 'aj'},
    '_type': 'foo'}],
  'max_score': 4.0445223,
  'total': 4},
 'timed_out': False,
 'took': 24}

# Assignment 


### Standard option
<p>Create a search engine for the <a href="http://data.politicalmashup.nl/arjan/telegraaf/">telegraaf newspaper collection</a> using  ElasticSearch. Make facets for years and document types.
            Pay attention to telephone numbers (in mini advertisements). Hieronder een voorbeeld van 1 document (= 1 artikeltje).
            <br/> Je ziet dat er zelfs een link naar de bron tekst (als plaatje) instaat. De URL linked door naar de nieuwe url <a href="http://www.delpher.nl/nl/kranten/view?identifier=ddd%3A010563762%3Ampeg21%3Aa0005&coll=ddd">http://www.delpher.nl/nl/kranten/view?identifier=ddd%3A010563762%3Ampeg21%3Aa0005&amp;coll=ddd</a>
            ElasticSearch gebruikt een JSON formaat als invoer, en dit is dus makkelijk om te zetten naar JSON.</p>
<pre>
&lt;pm:root>
&lt;pm:docinfo/>
&lt;pm:meta>
&lt;dc:date xmlns:dc="http://purl.org/dc/elements/1.1/">1923-03-01&lt;/dc:date>
&lt;dc:subject xmlns:dc="http://purl.org/dc/elements/1.1/">artikel&lt;/dc:subject>
&lt;dc:identifier xmlns:dc="http://purl.org/dc/elements/1.1/"
>ddd:010563762:mpeg21:p001:a0005&lt;/dc:identifier>
&lt;dc:source xmlns:dc="http://purl.org/dc/elements/1.1/">
&lt;dc:source>
&lt;pm:link pm:source="832675288" pm:description="De Telegraaf"/>
&lt;/dc:source>
&lt;/dc:source>
&lt;/pm:meta>
&lt;pm:content pm:source="http://kranten.kb.nl/view/article/id/ddd:010563762:mpeg21:p001:a0005"
pm:id="ddd:010563762:mpeg21:p001:a0005">
&lt;title pm:id="ddd:010563762:mpeg21:p001:a0005.t">De jongste maaregelen op den Rechter-
Rijn-oeven.&lt;/title>
&lt;text>
&lt;p pm:id="ddd:010563762:mpeg21:p001:a0005.1">â–  volgende redenen rijn bezet: lo. ter vereenvcudi-f
ging ran het douane-wezen en 2o. wegens fit' demonstratieÂ» en vergaderingen, welke in deÂ»e gebieden
zijn gehouden en gericht waren tegen de bezettingstroepen en de bezettingsautoriteiten. De
rijkscommissaris voor de bezette genie den heeft geweigerd, deze kennisgevins door te zenden. â€”
(Wolft},&lt;/p>
&lt;/text>
&lt;/pm:content>
&lt;/pm:root>
</pre>
   
### Other options
* Provided that you come up with a non trivial collection you can create a search engine using a different collection and also different software, as long as it is not done in MySQL.
* Discuss this with the assistants.
    * Convince us that your data set is interesting and your software solution worthwhile investigating.
* For the presentations it is fun to see something else.
* Usually it has a positive effect on your mark if you do something completely different ;-)
* You should do the same requirements.

#### Examples of other collections
* [Hillary Clinton Email collection](https://archive.org/details/hillary-clinton-emails-august-31-release) See also our <http://maartenmarx.nl/teaching/zoekmachines/Data/> folder.

<h2>Requirements</h2>
        <p>Each of the following points <strong>must</strong> be addressed. Create a seperate page on the wiki for each point. Make sure these pages can be found from the menu of your wiki. 
        Explain what you did, and exemplify with links to screenshots/a working system.</p>
        <ol>
        <li>Search as we know it from Google. Give a result page (SERP), with links to the documents and some description of each hit.</li>
            <li>Advanced search. Let a user be able to search in several fields, also in several fields simulteanously. Queries like "return articles with a title  about XXX  and which are   about YYY in the period ZZZ" should be possible.
             </li>
        <li>Do one of the following:
            <ol><li>Represent the hits of a query with a wordcloud of 25-50 informative words. The wordcloud should somehow summarise what the collection has to say about the query.
            You may think of these words as words that you could add to the query in order to improve recall (blind relevance feedback/query expansion). </li>
                <li>Represent each document   with a word-cloud. Also make word-clouds for the question and for the answer.
            EXAMPLE: The html files in <a href="http://data.politicalmashup.nl/arjan/odeii/data_as_html/">http://data.politicalmashup.nl/arjan/odeii/data_as_html/</a> contain such wordcloud summaries, which work rather well.</li></ol>
        <br/>You can use several techniques to get rid of high frequency, but meaningless words: of course IDF, but also mutual information (see 13.5.1), or of course the technique from the paper by Kaptein et al on wordclouds.
        </li>
          <!--  <li>Group (nearly) identical documents. It is bad if two or more of the precious places of your top ten hits are occupied by near identical copies of a document.
            Even more so if it is not relevant! So, get rid of these, by grouping them for instance. You get an idea of these duplicates if you search for "neuken".</li>
            -->
            <li>Give next to a traditional list of results, a timeline in which you indicate how many hits there are over time.</li>
            <li>Provide Faceted Search next to the traditional list of results. For the "Telegraaf" collectie, use the <tt>dc:subject</tt> element as facet values.</li>
            <li><strong>Evaluate your results</strong> Let 2 persons assess the relevancy of the top 10 documents for <strong>5 different queries</strong>. Compute Cohen's kappa. Determine the average precision at 10 for your system based on these 10 queries, and the two relevance assesments. 
                Also plot the P@10 (for both judges) for each query, showing differences in hard and easy queries.  
                Describe clearly how you solved differences in judgements.
            <br/>
            Create your queries in the following format:
<pre>
&lt;topic number="6"  >
    &lt;query>kcs&lt;/query>
    &lt;description>Find information on the Kansas City Southern railroad.
    &lt;/description>

&lt;/topic>

&lt;topic number="16"  >
    &lt;query>arizona game and fish&lt;/query>
    &lt;description>I'm looking for information about fishing and hunting
    in Arizona.
    &lt;/description>
&lt;/topic>
</pre>
    So, both provide the actual query, and a description of the information need that was behind the query.
    <br/>
    Give a small set of clear guidelines for judging the results, and let your judges follow these guidelines.
    <br/>
    It is far more interesting to have difficult queries (both for the search engine and for the judges) than to have queries on which all ten retrieved documents are relevant.
    So, try to create a good list of information needs.

</li>
<li>Change the ranking of your system, compute the average precision at 10 using your 10 queries, compare the results to your old system, and EXPLAIN what is going on.</li>
</ol>

<h2>Presentation</h2>
<p>During your presentation you should have a live working search engine, that you demonstrate on the spot. Your presentation should be structured so that you will show all  requirements.
You will be asked to show how your system works using information needs coming  from the audience.</p>
<p>**Hint:** focus on a special aspect of your project. Everyone has done something similar, so your audience knows what was hard and what was terrible. Pick something you think will interest them.</p>


<h2>How you will be marked</h2>
<ul>
    <li>Sent the URL of your guthub wiki to Maarten Marx BEFORE the presentation.</li>
    <li>The first page of the wiki should contain:
        <ol>
            <li>The names and student numbers of the project members</li>
            <li>A link to the slides of your presentation</li>
            <li>A table of contents, with links to pages on the wiki adressing one of each of the "must-have" points listed above.
            <br/>During grading, you will receive points for each of the points. So make it crystal clear where they are adressed in your wiki. Use one page per point.</li>


</ol>
</li>
<li>The page for each point should contain, all rather briefly,
<ol>
<li>What you did and why you choose to do it in your special way.</li>
<li>Examples of what works, and what does not work (very well).</li>
<li>An evaluation of the quality of your work in 3-4 sentences.</li>
</ol></li>
<li>Clickable links to a live working demo are <strong>highly appreciated</strong>. </li>
</ul>